In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import networkx as nx
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
import tensorflow as tf
from IPython.display import YouTubeVideo
plt.style.use('ggplot')

2024-02-18 16:21:03.554455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(os.listdir("/Users/runzhou/Desktop/rag/rag/data/yt8m/frame/"))

['2_frame_train_download_plan.json', 'train0274.tfrecord', 'train0580.tfrecord', 'train2161.tfrecord']


In [4]:
# Explore data
video_lvl_data_path = "/Users/runzhou/Desktop/rag/rag/data/yt8m/video/train0274.tfrecord"
raw_dataset = tf.data.TFRecordDataset(video_lvl_data_path)


vid_ids = []
labels = []
rgb = []
audio = []

for example in tf.compat.v1.python_io.tf_record_iterator(video_lvl_data_path):
    seq_example = tf.train.Example.FromString(example)
    vid_ids.append(seq_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(seq_example.features.feature['labels'].int64_list.value)
    rgb.append(seq_example.features.feature['mean_rgb'].float_list.value)
    audio.append(seq_example.features.feature['mean_audio'].float_list.value)



Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [5]:
print('Number of videos in this tfrecord: ',len(vid_ids))
print ('Number of labels in this tfrecord: ', len (labels))
print('Picking a youtube video id:', vid_ids[1])

Number of videos in this tfrecord:  1045
Number of labels in this tfrecord:  1045
Picking a youtube video id: zseA


In [7]:
import requests

def translate_video_id(encoded_id):
    # Construct the URL to retrieve the mapping
    url = f"http://data.yt8m.org/2/j/i/{encoded_id[:2]}/{encoded_id}.js"
    response = requests.get(url)

    # Check if the response is successful and has the expected format
    if response.status_code == 200 and 'i("' in response.text:
        # Extract the standard YouTube video ID from the response
        video_id = response.text.split('"')[3]
        return video_id
    else:
        # Log an error or handle it as appropriate
        print(f"Error retrieving video ID for {encoded_id}: {response.text}")
        return '200'

youtube_urls = []
for encoded_id in vid_ids:
    video_id = translate_video_id(encoded_id)
    if video_id:
        youtube_url = f"https://www.youtube.com/watch?v={video_id}"
        youtube_urls.append(youtube_url)

# Print or use the YouTube URLs
for url in youtube_urls:
    print(url)



Error retrieving video ID for CteA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for r3eA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for xleA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for lveA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for I1eA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for tKeA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Message>Access denied.</Message></Error>
Error retrieving video ID for dfeA: <?xml version='1.0' encoding='UTF-8'?><Error><Code>AccessDenied</Code><Messa

In [8]:
# Read in vocabulary for human-rated mapping of numeric label to text label for the video
vocab = pd.read_csv('./data/vocabulary.csv')

# Sanity checks on the labels, youtube video content
for i in range(0, 5):
    text_labels = [vocab[vocab['Index'] == i]['Name'].values[0] for i in labels[i]]
    
    print("Label: ", labels[i], " | ", "Video: ", youtube_urls[i], " | ", text_labels)

Label:  [14]  |  Video:  https://www.youtube.com/watch?v=200  |  ['Music video']
Label:  [14]  |  Video:  https://www.youtube.com/watch?v=-4Ms2Fc_JoE  |  ['Music video']
Label:  [0, 12, 284, 3754]  |  Video:  https://www.youtube.com/watch?v=ox2hNpnqF9o  |  ['Game', 'Association football', 'GoPro', 'Indoor soccer']
Label:  [15, 18, 67, 140, 1785]  |  Video:  https://www.youtube.com/watch?v=egf8Rc4EkSU  |  ['Animal', 'Pet', 'Dog', 'Puppy', 'Poodle']
Label:  [2, 45, 295]  |  Video:  https://www.youtube.com/watch?v=eg9CRKtCiy8  |  ['Vehicle', 'Train', 'Train station']
